In [24]:
from fastapi import FastAPI
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

API

In [25]:
#app = FastAPI()


In [26]:
#Cargo el Dataset y lo convierto en un df
df_api=pd.read_parquet(r'C:\Users\Jorge\OneDrive\Escritorio\Proyecto 1\Data_Api.parquet')

In [27]:
df_api.head()

,id,title,release_date,release_year,cast_members,director,vote_average,vote_count,budget,revenue,return,popularity
0,862,Toy Story,1995-10-30,1995,"[Tom Hanks, Tim Allen, Don Rickles, Jim Varney...",John Lasseter,7.7,5415.0,30000000,373554033.0,12.451801,21.946943
1,8844,Jumanji,1995-12-15,1995,"[Robin Williams, Jonathan Hyde, Kirsten Dunst,...",Joe Johnston,6.9,2413.0,65000000,262797249.0,4.043035,17.015539
2,15602,Grumpier Old Men,1995-12-22,1995,"[Walter Matthau, Jack Lemmon, Ann-Margret, Sop...",Howard Deutch,6.5,92.0,0,0.0,0.000000,11.712900
3,31357,Waiting to Exhale,1995-12-22,1995,"[Whitney Houston, Angela Bassett, Loretta Devi...",Forest Whitaker,6.1,34.0,16000000,81452156.0,5.090760,3.859495
4,11862,Father of the Bride Part II,1995-02-10,1995,"[Steve Martin, Diane Keaton, Martin Short, Kim...",Charles Shyer,5.7,173.0,0,76578911.0,0.000000,8.387519


In [28]:
#1. cantidad_filmaciones_mes( Mes ): Se ingresa un mes en idioma Español. Debe devolver la cantidad de películas que fueron estrenadas en el mes consultado en la totalidad del dataset.
#Ejemplo de retorno: X cantidad de películas fueron estrenadas en el mes de X

#@app.get('/cantidad_filmaciones_mes/{mes}')
def cantidad_filmaciones_mes(mes: str):
    
    #Creo un diccionario con los meses y el número que le corresponde
    meses_numero = {"enero": 1, "febrero": 2, "marzo": 3, "abril": 4, "mayo": 5, "junio": 6,"julio": 7, "agosto": 8, "septiembre": 9, "octubre": 10, "noviembre": 11, "diciembre": 12 }
    
    #Paso a minusculas para evitar discrepancias tipo Junio != junio
    mes=mes.lower()
    #Validacion de entrada
    if mes not in meses_numero:
        return f'Error: Mes invalido. Intentalo de nuevo con un mes en español. Checa la ortografía'
    #Cambio el mes a numero 
    mes_num=meses_numero[mes]
    
    #Filtrado y conteo del mes en el df
    total=df_api[df_api['release_date'].dt.month==mes_num].shape[0]
    return f'{total} películas fueron estrenadas en el mes de {mes}'

    



In [29]:
cantidad_filmaciones_mes('JUnio')

'3098 películas fueron estrenadas en el mes de junio'

In [30]:
#2.def cantidad_filmaciones_dia( Dia ): Se ingresa un día en idioma Español. Debe devolver la cantidad de películas que fueron estrenadas en día consultado en la totalidad del dataset.
#Ejemplo de retorno: X cantidad de películas fueron estrenadas en los días X

#@app.get('/cantidad_filmaciones_dia/{dia}')

def cantidad_filmaciones_dia(dia: str):

    #Creo diccionario con los dias y sus números
    dia_numero={"lunes": 0, "martes": 1, "miércoles": 2, "jueves": 3, "viernes": 4, "sábado": 5, "domingo": 6}

    #Paso a minusculas
    dia=dia.lower()
    #Validación de entrada
    if dia not in dia_numero:
        return f'Error: Día invalido. Intentalo de nuevo con un día en español. Checa la ortografía'
    #Cambio a numero
    dia_num=dia_numero[dia]

    #Filtrado y conteo 
    total=df_api[df_api['release_date'].dt.dayofweek==dia_num].shape[0]
    return f'{total} películas fueron estrenadas en el día {dia}'


In [31]:
cantidad_filmaciones_dia('lunes')

'3415 películas fueron estrenadas en el día lunes'

In [32]:
#3.def score_titulo( titulo_de_la_filmación ): Se ingresa el título de una filmación esperando como respuesta el título, el año de estreno y el score.
#Ejemplo de retorno: La película X fue estrenada en el año X con un score/popularidad de X

#@app.get('/score_titulo/{titulo}')
def score_titulo(titulo: str):

    #Para evitar errores de ortografia lo estandarizamos
    titulo = titulo.lower()

    #Buscamos el titulo en el df y tambien lo pasamos a minusculas
    movie=df_api[df_api['title'].str.lower()==titulo]


    #Validacion
    if movie.empty:
        raise HTTPException(status_code=404, detail="Título no encontrado")
    
    #Obtengo los datos
    movie=movie.iloc[0]
    nombre=movie['title']
    año=movie['release_year']
    score=movie['popularity']

    return f'La película {nombre} fue estrenada en el año {año} con un score/popularidad de {score}'


In [33]:
score_titulo('Father of the Bride Part II')

'La película Father of the Bride Part II fue estrenada en el año 1995 con un score/popularidad de 8.387519'

In [34]:
#4. def votos_titulo( titulo_de_la_filmación ): Se ingresa el título de una filmación esperando como respuesta el título, la cantidad de votos y el valor promedio de las votaciones. La misma variable deberá de contar con al menos 2000 valoraciones, caso contrario, debemos contar con un mensaje avisando que no cumple esta condición y que por ende, no se devuelve ningun valor.
#Ejemplo de retorno: La película X fue estrenada en el año X. La misma cuenta con un total de X valoraciones, con un promedio de X

#@app.get("/votos_titulo/{titulo}")
def votos_titulo(titulo: str):

    #Para evitar errores de ortografia lo estandarizamos
    titulo = titulo.lower()
    
    #Buscamos el titulo en el df y tambien lo pasamos a minusculas
    movie = df_api[df_api['title'].str.lower() == titulo.lower()]

    #Validacion
    if movie.empty:
        raise HTTPException(status_code=404, detail="Título no encontrado")
    
    #Obtengo los datos
    movie=movie.iloc[0]
    nombre=movie['title']
    año=movie['release_year']
    votos=movie['vote_count']
    promedio=movie['vote_average']

    return f'La película {nombre} fue estrenada en el año {año}. La misma cuenta con un total de {votos} valoraciones, con un promedio de {promedio}'
    



In [35]:
votos_titulo('Father of the Bride Part II')

'La película Father of the Bride Part II fue estrenada en el año 1995. La misma cuenta con un total de 173.0 valoraciones, con un promedio de 5.7'

In [36]:
#5. def get_actor( nombre_actor ): Se ingresa el nombre de un actor que se encuentre dentro de un dataset debiendo devolver el éxito del mismo medido a través del retorno. Además, la cantidad de películas que en las que ha participado y el promedio de retorno. La definición no deberá considerar directores.
#Ejemplo de retorno: El actor X ha participado de X cantidad de filmaciones, el mismo ha conseguido un retorno de X con un promedio de X por filmación

#@app.get("/get_actor/{nombre_actor}")
def get_actor(nombre_actor: str):

    #Para evitar errores de ortografia lo estandarizamos
    nombre_actor=nombre_actor.lower()

    #Busco las peliculas en las que aparece como actor y lo estandarizamos
    movies=df_api[df_api['cast_members'].apply(lambda x:nombre_actor in [y.lower() for y in x])]

    #Validacion
    if movies.empty:
        raise HTTPException(status_code=404, detail='Actor no encontrado')

    #Obtengo los datos 
    cantidad_filmaciones=movies.shape[0]
    retorno=round(movies['return'].sum(),2)
    retorno_promedio=round((retorno/cantidad_filmaciones),2)

    return f'El actor {nombre_actor} ha participado en {cantidad_filmaciones} películas, el mismo ha conseguido un retorno de {retorno} con un promedio de {retorno_promedio} por filmación'



    



    

In [37]:
get_actor('Tom Hanks')

'El actor tom hanks ha participado en 70 películas, el mismo ha conseguido un retorno de 178.85 con un promedio de 2.55 por filmación'

In [38]:
#6.def get_director( nombre_director ): Se ingresa el nombre de un director que se encuentre dentro de un dataset debiendo devolver el éxito del mismo medido a través del retorno.
# Además, deberá devolver el nombre de cada película con la fecha de lanzamiento, retorno individual, costo y ganancia de la misma.


#@app.get("/get_director/{nombre_director}")
def get_director(nombre_director: str):
    df = load_data()
    films = df[df['director'].str.contains(nombre_director, case=False, na=False)]
    if films.empty:
        return {"error": "Director no encontrado"}
    film_details = films[['title', 'release_date', 'return', 'budget', 'revenue']].to_dict(orient='records')
    return {
        "director": nombre_director,
        "peliculas": film_details}

In [39]:
def get_director(nombre_director: str):

    #Para evitar errores de ortografia lo estandarizamos
    nombre_director=nombre_director.lower()


    #Buscamos las peliculas que ha dirigido y lo estandarizamos
    movies=df_api[df_api['director'].str.lower()==nombre_director]

    #Validacion
    if movies.empty:
        raise HTTPException(status_code=404, detail='Director no encontrado')
    
    #Obtengo los datos 
    retorno=round(movies['return'].sum(),2)
    peliculas_dirigidas=movies[['title', 'release_date', 'return', 'budget', 'revenue']].to_dict(orient='records')
    cantidad_filmaciones=movies.shape[0]

    return {'Director':nombre_director,
            'Cantidad de Películas':cantidad_filmaciones,
            'Retorno total': retorno,
            'Películas':peliculas_dirigidas}

    

In [40]:
get_director('John Lasseter')

{'Director': 'john lasseter',
 'Cantidad de Películas': 10,
 'Retorno total': 27.65,
 'Películas': [{'title': 'Toy Story',
   'release_date': Timestamp('1995-10-30 00:00:00'),
   'return': 12.4518011,
   'budget': 30000000,
   'revenue': 373554033.0},
  {'title': "A Bug's Life",
   'release_date': Timestamp('1998-11-25 00:00:00'),
   'return': 3.0271571583333334,
   'budget': 120000000,
   'revenue': 363258859.0},
  {'title': 'Toy Story 2',
   'release_date': Timestamp('1999-10-30 00:00:00'),
   'return': 5.526298544444445,
   'budget': 90000000,
   'revenue': 497366869.0},
  {'title': 'Luxo Jr.',
   'release_date': Timestamp('1986-08-17 00:00:00'),
   'return': 0.0,
   'budget': 0,
   'revenue': 0.0},
  {'title': 'Cars',
   'release_date': Timestamp('2006-06-08 00:00:00'),
   'return': 3.849859575,
   'budget': 120000000,
   'revenue': 461983149.0},
  {'title': 'Cars 2',
   'release_date': Timestamp('2011-06-11 00:00:00'),
   'return': 2.79926198,
   'budget': 200000000,
   'revenue':

Modelo

def recomendacion( titulo ): Se ingresa el nombre de una película y te recomienda las similares en una lista de 5 valores.

In [41]:
df_modelo=pd.read_parquet(r'C:\Users\Jorge\OneDrive\Escritorio\Proyecto 1\Data_Modelo')

In [63]:
# Crear una instancia de TfidfVectorizer con las stopwords en inglés
tfidf = TfidfVectorizer(stop_words='english')
# Crear la matriz tf-idf con los features de las películas
tfidf_matrix = tfidf.fit_transform(df_modelo['genres'] + ' ' + df_modelo['overview'] + ' ' + df_modelo['cast_members'] + ' ' + df_modelo['director'])

In [69]:
#@app.get("/recomendacion/{title}")
def recomendacion(title: str):
    
    #Validacion de inicio
    if title not in df_modelo["title"].values:
           return {"Error": "Título no encontrado"}
    
    idx = df_modelo[df_modelo['title'] == title].index[0]
    #Calculo de la similitud del coseno
    cosine_sim = cosine_similarity(tfidf_matrix[idx:idx+1], tfidf_matrix).flatten()
    #Recopilacion de los scores de similitud en una lista de tuplas, donde el primer elemento es el índice y el segundo es el score.
    sim_scores = list(enumerate(cosine_sim))
    #Ordenar la lista de mayor a menor.
    sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)
    # Obtener los 5 ítems más similares
    sim_scores = sim_scores[:6]

     #Excluir la primera película si es la misma que se ingresó
    if sim_scores[0][0] == idx:
        sim_scores = sim_scores[1:6]
    else:
        sim_scores = sim_scores[:5]
    # Obtener los títulos de los ítems recomendados y convertirlos en lista
    recommended_movies = df_modelo['title'].iloc[[i[0] for i in sim_scores]].tolist()

    result = {"Películas recomendadas": recommended_movies}

    # Devolver las películas recomendadas
    return result

In [70]:
recomendacion('Jumanji')

{'Películas recomendadas': ['Wreck-It Ralph',
  'Rhinoceros',
  'Turkey Shoot',
  'Word Wars',
  'Quiz']}